<a href="https://colab.research.google.com/github/markriis/IDS-pollution-of-cities/blob/main/Data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing data and librarys

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

!pip install -q opendatasets
import opendatasets as od


In [8]:
#air_and_water = pd.read_csv('https://raw.githubusercontent.com/markriis/IDS-pollution-of-cities/main/cities_air_quality_water_pollution2021.csv')

gasses = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv',
                     usecols=['country', 'year', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_change_co2_per_capita', 'methane',
       'methane_per_capita', 'nitrous_oxide', 'nitrous_oxide_per_capita',
       'oil_co2', 'oil_co2_per_capita', 'other_co2_per_capita',
       'other_industry_co2', 'primary_energy_consumption',
       'total_ghg', 'total_ghg_excluding_lucf'])# Removed some columns that I doubt would be of use, meanings at https://github.com/owid/co2-data/blob/master/owid-co2-codebook.csv

air = pd.read_excel('https://cdn.who.int/media/docs/default-source/air-pollution-documents/air-quality-and-health/who_aap_2021_v9_11august2022.xlsx?sfvrsn=9035996c_3',
                    storage_options={'User-Agent': 'Mozilla/5.0'}, sheet_name='AAP_2022_city_v9',
                    usecols=['WHO Country Name', 'City or Locality', 'Measurement Year', 'PM2.5 (μg/m3)', 'PM10 (μg/m3)', 'NO2 (μg/m3)',
                             'PM25 temporal coverage (%)', 'PM10 temporal coverage (%)', 'NO2 temporal coverage (%)', 'Version of the database'])

development = pd.read_csv('https://hdr.undp.org/sites/default/files/2021-22_HDR/HDR21-22_Composite_indices_complete_time_series.csv',
                          storage_options={'User-Agent': 'Mozilla/5.0'},
                          usecols=['country', 'hdi_1990', 'hdi_1991', 'hdi_1992', 'hdi_1993',
       'hdi_1994', 'hdi_1995', 'hdi_1996', 'hdi_1997', 'hdi_1998',
       'hdi_1999', 'hdi_2000', 'hdi_2001', 'hdi_2002', 'hdi_2003',
       'hdi_2004', 'hdi_2005', 'hdi_2006', 'hdi_2007', 'hdi_2008',
       'hdi_2009', 'hdi_2010', 'hdi_2011', 'hdi_2012', 'hdi_2013',
       'hdi_2014', 'hdi_2015', 'hdi_2016', 'hdi_2017', 'hdi_2018',
       'hdi_2019', 'hdi_2020', 'hdi_2021'])#

dataset_url = 'https://public.opendatasoft.com/explore/dataset/geonames-all-cities-with-a-population-1000/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B'
od.download(dataset_url)# This one takes a while

populations = pd.read_csv('./geonames-all-cities-with-a-population-1000.csv', delimiter=';',
                          usecols=['Name', 'ASCII Name', 'Country name EN', 'Population', 'Modification date'],
                          parse_dates=['Modification date'])

26992640it [00:58, 462688.84it/s]


In [25]:
#air_and_water.head()

In [10]:
air.head()

,WHO Country Name,City or Locality,Measurement Year,PM2.5 (μg/m3),PM10 (μg/m3),NO2 (μg/m3),PM25 temporal coverage (%),PM10 temporal coverage (%),NO2 temporal coverage (%),Version of the database
0,Afghanistan,Kabul,2019,119.77,NaN,NaN,18.0,NaN,NaN,2022
1,Albania,Durres,2015,NaN,17.65,26.63,NaN,NaN,83.961187,2022
2,Albania,Durres,2016,14.32,24.56,24.78,NaN,NaN,87.932605,2022
3,Albania,Elbasan,2015,NaN,NaN,23.96,NaN,NaN,97.853881,2022
4,Albania,Elbasan,2016,NaN,NaN,26.26,NaN,NaN,96.049636,2022


In [11]:
development.head()

,country,hdi_1990,hdi_1991,hdi_1992,hdi_1993,hdi_1994,hdi_1995,hdi_1996,hdi_1997,hdi_1998,...,hdi_2012,hdi_2013,hdi_2014,hdi_2015,hdi_2016,hdi_2017,hdi_2018,hdi_2019,hdi_2020,hdi_2021
0,Afghanistan,0.273,0.279,0.287,0.297,0.292,0.310,0.319,0.323,0.324,...,0.466,0.474,0.479,0.478,0.481,0.482,0.483,0.488,0.483,0.478
1,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.541,0.552,0.563,0.582,0.596,0.597,0.595,0.595,0.590,0.586
2,Albania,0.647,0.629,0.614,0.617,0.624,0.634,0.645,0.642,0.657,...,0.778,0.785,0.792,0.795,0.798,0.802,0.806,0.810,0.794,0.796
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.869,0.864,0.871,0.867,0.871,0.868,0.872,0.873,0.848,0.858
4,United Arab Emirates,0.728,0.739,0.742,0.748,0.755,0.762,0.767,0.773,0.779,...,0.846,0.852,0.859,0.865,0.870,0.897,0.909,0.920,0.912,0.911


In [12]:
gasses.head()

,country,year,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_including_luc,co2_including_luc_growth_abs,co2_including_luc_growth_prct,...,nitrous_oxide_per_capita,oil_co2,oil_co2_per_capita,other_co2_per_capita,other_industry_co2,primary_energy_consumption,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1850,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1851,3767956.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1852,3783940.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1853,3800954.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1854,3818038.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
gasses.shape

(50598, 53)

In [26]:
populations.head()

,Name,ASCII Name,Country name EN,Population,Modification date
0,Fiumicino,Fiumicino,Italy,34556,2022-11-17
1,Figline Valdarno,Figline Valdarno,Italy,13287,2019-10-24
2,Ficarolo,Ficarolo,Italy,2007,2014-04-13
3,Ferrara di Monte Baldo,Ferrara di Monte Baldo,Italy,89,2014-04-13
4,Felizzano,Felizzano,Italy,2276,2014-01-11


Finding what countries are in all dataframes

In [ ]:
countries_air = air['WHO Country Name'].dropna().values
countries_dev = development['country'].dropna().values
countries_gas = gasses['country'].dropna().values
countries_pop = populations['Country name EN'].dropna().values

display(pd.unique(countries_air).shape)# It apears air has the least countries in it, but still 118 is prety good
display(pd.unique(countries_dev).shape)
display(pd.unique(countries_gas).shape)
display(pd.unique(countries_pop).shape)

In [17]:
common_countries = np.intersect1d(np.intersect1d(countries_air, countries_dev), np.intersect1d(countries_gas, countries_pop))
display(common_countries)
display(common_countries.shape)

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Argentina',
       'Australia', 'Austria', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Belgium', 'Bhutan', 'Bosnia and Herzegovina', 'Brazil',
       'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Denmark', 'Ecuador',
       'Egypt', 'El Salvador', 'Estonia', 'Ethiopia', 'Fiji', 'Finland',
       'France', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iraq',
       'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan',
       'Kazakhstan', 'Kenya', 'Kuwait', 'Kyrgyzstan', 'Latvia', 'Lebanon',
       'Liechtenstein', 'Lithuania', 'Luxembourg', 'Madagascar',
       'Malaysia', 'Maldives', 'Malta', 'Mauritius', 'Mexico', 'Mongolia',
       'Montenegro', 'Morocco', 'Myanmar', 'Nepal', 'Netherlands',
       'New Zealand', 'Nigeria', 'Norway', 'Pakistan', 'Panama',
       'Paraguay', 'Peru', 'P

(106,)

In [28]:
air.rename(columns={'WHO Country Name': 'country'}, inplace=True)
populations.rename(columns={'Country name EN': 'country'}, inplace=True)

datasets = {
    'gasses' : gasses,
    'air' : air,
    'development' : development,
    'populations' : populations
}

In [29]:
for name, dataset in datasets.items():
  datasets[name] = dataset[dataset['country'].isin(common_countries)]

In [32]:
datasets['gasses'] = datasets['gasses'][datasets['gasses']['year'] >= 1990]

In [33]:
for name, dataset in datasets.items():
  display(dataset.shape)

(3392, 53)

(29197, 10)

(106, 33)

(110603, 5)

In [22]:
import sys

sys.getsizeof(populations)

39471843